In [ ]:
!pip install stanza

from pathlib import Path
from itertools import product
from operator import itemgetter
import nltk
import math
import operator
import stanza

nltk.download('punkt')
nltk.download('wordnet')
stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize')


In [ ]:
def sentence_to_bow(text, stopwords=None):
  
    bow = set()
    punct = {'.',',', ';', '(', ')', '{', '}', '[', ']', "’", '‘',  '“', '”',':', '?', '!'} 
    lemmatizer = nltk.WordNetLemmatizer()

    for sentence in nltk.sent_tokenize(text): # sentence granularity tokenization
        sentence = sentence.lower()
        sentence_bow = set(nltk.word_tokenize(sentence)) # word granularity tokenization
        
        sentence_bow = sentence_bow.difference(punct)
        
        if stopwords:
            sentence_bow = sentence_bow.difference(stopwords)

        bow.update(sentence_bow)

    return set(lemmatizer.lemmatize(token) for token in bow) # lemmatization

In [ ]:
#creo la mappa di Nasari
nasari_map = {}

with Path('/content/dd-small-nasari-15.txt').open('r') as file:
    for i,line in enumerate(file.readlines()):

        parsed_line = line.split(';')
        babel_synset = parsed_line[1].lower()
        nasari_vector = []
        #print(parsed_line)
        for chunk in parsed_line[2:]:
            #print(chunk)
            if len(chunk) > 1:
                array = chunk.split('_')
                if len(array) == 2:
                    lemma = array[0]
                    score = array[1]
                    nasari_vector.append((lemma.lower(), float(score))) # normalize text with lower()
                        
        nasari_map[babel_synset] = nasari_vector # build dictionary for each line of the file

#creo array stopwords
stop_words = set(Path('/content/Stop-Words.txt').open('r').read().splitlines())



In [ ]:
def weighted_overlap(v1, v2):

        v1_components = set([elem[0] for elem in v1]) 
        v2_components = set([elem[0] for elem in v2])

        # components shared by both vectors
        overlapped_components = v1_components.intersection(v2_components) 
        
        wo = 0
        if len(overlapped_components) > 0:
            num = sum([1.0/(rankWO(q,v1) + rankWO(q,v2)) for q in overlapped_components])
            den = sum([1.0/(2*(i+1)) for i,_ in enumerate(overlapped_components)])
            wo = num / den
        
        return math.sqrt(wo)

In [ ]:
def rankWO(component, vector):

        # vector ordered by score
        ranking = sorted(vector, key=lambda v: v[1], reverse=True) # descending order by score
        
        rank_pos= 0
        for pos, (lemma, _) in enumerate(vector, start=1):
            if lemma == component:
                rank_pos = pos
                break
        
        return rank_pos

In [ ]:
############### CHUNKS #################

#individuo il topic del titolo

with Path('/content/Ebola-virus-disease.txt').open('r') as file:
    for i,line in enumerate(file.readlines()):
        if i == 3:
            title = line


topic = []
title_bow = sentence_to_bow(title, stop_words)

for word in title_bow:
    if word in nasari_map:
        topic.append(nasari_map[word])


#estraggo i topic dai vari chunk

body = []
topic_chunk = []
topic_body = []

with Path('/content/Ebola-virus-disease.txt').open('r') as file:
    source =  next(file)[2:].strip() # jump '# ' chars 
    next(file) # empty line after title
    body = [line.strip() for line in file if line.strip() != ''] # paragraph level segmentation

i = 0

for chunk in body:
    print(i)
    print(chunk)

    for word in sentence_to_bow(chunk, stopwords=stop_words):
    
        if word in nasari_map:
            topic_chunk.append(nasari_map[word])
    
    topic_body.append(topic_chunk)    
    topic_chunk = []
    i += 1


# weighted overlap over chunks

ranking_chunks = {}
j = 0

for tc in topic_body:
    
    chunk_sim = 0
    n = 0
    for v1, v2 in product(tc, topic):
        chunk_sim += weighted_overlap(v1,v2)
        n += 1

    if n != 0:
        ranking_chunks[j] = chunk_sim / n
    else:
        ranking_chunks[j] = 0

    j += 1


sorted_ranking = dict(sorted(ranking_chunks.items(), key=operator.itemgetter(1),reverse=True))

######################################### SELECT COMPRESSION RATE ####################################################
compression_rate = 0.5

selected_chunks = list(map(itemgetter(0), sorted_ranking.items()))[0:int(len(sorted_ranking)*compression_rate)]


i = 0
f = open("compressed_chunk.txt", "a")

print("\n__________________________________________________________ " + str(len(selected_chunks)) + "/" + str(len(body)) + " __________________________________________________________\n")
for chunk in body:
    if i in selected_chunks:
        print(i)
        print(chunk)
        f.write("\n" + chunk)

    i +=1

f.close()

In [ ]:
############### SENTENCE #################

#individuo il topic del titolo

with Path('/content/Ebola-virus-disease.txt').open('r') as file:
    for i,line in enumerate(file.readlines()):
        if i == 3:
            title = line


topic = []
title_bow = sentence_to_bow(title, stop_words)

for word in title_bow:
    if word in nasari_map:
        topic.append(nasari_map[word])


#estraggo i topic dalle frasi

body = []
topic_chunk = []
topic_body = []

with Path('/content/Ebola-virus-disease.txt').open('r') as file:
    source =  next(file)[2:].strip() # jump '# ' chars 
    next(file) # empty line after title
    body = [line.strip() for line in file if line.strip() != ''] 




sentences = []

for x in body:
    doc = nlp(x)
    for sentence in doc.sentences:
        sentences.append(sentence.text)


i = 0

for sentece in sentences:
    print(i)
    print(sentece)

    for word in sentence_to_bow(sentece, stopwords=stop_words):
    
        if word in nasari_map:
            topic_chunk.append(nasari_map[word])
    
    topic_body.append(topic_chunk)    
    topic_chunk = []
    i += 1


# weighted overlap over sentences

ranking_chunks = {}
j = 0

for tc in topic_body:
    
    chunk_sim = 0
    n = 0
    for v1, v2 in product(tc, topic):
        chunk_sim += weighted_overlap(v1,v2)
        n += 1

    if n != 0:
        ranking_chunks[j] = chunk_sim / n
    else:
        ranking_chunks[j] = 0

    j += 1


sorted_ranking = dict(sorted(ranking_chunks.items(), key=operator.itemgetter(1),reverse=True))

######################################### SELECT COMPRESSION RATE ####################################################
compression_rate = 0.5

selected_chunks = list(map(itemgetter(0), sorted_ranking.items()))[0:int(len(sorted_ranking)*compression_rate)]


i = 0
f = open("compressed_sentence.txt", "a")


print("\n__________________________________________________________ " + str(len(selected_chunks)) + "/" + str(len(sentences)) + " __________________________________________________________\n")

for chunk in sentences:
    if i in selected_chunks:
        print(i)
        print(chunk)
        f.write("\n" + chunk)

    i +=1

f.close()

0
Ebola virus disease
1
Ebola virus disease (EVD), or simply Ebola, is a viral haemorrhagic fever of humans and other primates caused by ebolaviruses.
2
Signs and symptoms typically start between two days and three weeks after contracting the virus with a fever, sore throat, muscular pain, and headaches.
3
Vomiting, diarrhoea and rash usually follow, along with decreased function of the liver and kidneys.
4
At this time, some people begin to bleed both internally and externally.
5
The disease has a high risk of death, killing 25% to 90% of those infected, with an average of about 50%.
6
This is often due to low blood pressure from fluid loss, and typically follows six to 16 days after symptoms appear.
7
The virus spreads through direct contact with body fluids, such as blood from infected humans or other animals.
8
Spread may also occur from contact with items recently contaminated with bodily fluids.
9
Spread of the disease through the air between primates, including humans, has not b